In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jofgetXqPPBxZ9HDeYpRH87wrmKIPt_CgHc2vd92LVPQ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'aiagent-donotdelete-pr-hhlreaqgqptmqx'
object_key = 'ICRISAT-District Level Data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,Dist Code,Year,State Code,State Name,Dist Name,RICE AREA (1000 ha),RICE PRODUCTION (1000 tons),RICE YIELD (Kg per ha),WHEAT AREA (1000 ha),WHEAT PRODUCTION (1000 tons),...,SUGARCANE YIELD (Kg per ha),COTTON AREA (1000 ha),COTTON PRODUCTION (1000 tons),COTTON YIELD (Kg per ha),FRUITS AREA (1000 ha),VEGETABLES AREA (1000 ha),FRUITS AND VEGETABLES AREA (1000 ha),POTATOES AREA (1000 ha),ONION AREA (1000 ha),FODDER AREA (1000 ha)
0,1,1966,14,Chhattisgarh,Durg,548.0,185.0,337.59,44.0,20.0,...,1777.78,0.0,0.0,0.0,5.95,6.64,12.59,0.01,0.60,0.47
1,1,1967,14,Chhattisgarh,Durg,547.0,409.0,747.71,50.0,26.0,...,1500.00,0.0,0.0,0.0,5.77,7.24,13.02,0.01,0.56,1.23
2,1,1968,14,Chhattisgarh,Durg,556.3,468.0,841.27,53.7,30.0,...,1000.00,0.0,0.0,0.0,5.41,7.40,12.81,0.10,0.58,1.02
3,1,1969,14,Chhattisgarh,Durg,563.4,400.8,711.40,49.4,26.5,...,1900.00,0.0,0.0,0.0,5.52,7.16,12.69,0.01,0.56,0.84
4,1,1970,14,Chhattisgarh,Durg,571.6,473.6,828.55,44.2,29.0,...,2000.00,0.0,0.0,0.0,5.45,7.19,12.64,0.02,0.52,0.42
5,1,1971,14,Chhattisgarh,Durg,581.8,412.9,709.69,44.4,25.8,...,2000.00,0.0,0.0,0.0,5.42,7.48,12.91,0.01,0.54,0.38
6,1,1972,14,Chhattisgarh,Durg,582.2,381.0,654.41,39.6,20.6,...,2000.00,0.0,0.0,0.0,5.48,7.53,13.01,0.01,0.55,0.26
7,1,1973,14,Chhattisgarh,Durg,600.0,471.9,786.50,37.3,18.6,...,6000.00,0.0,0.0,0.0,5.30,7.60,12.90,0.01,0.53,0.14
8,1,1974,14,Chhattisgarh,Durg,587.4,219.0,372.83,36.5,22.4,...,1875.00,0.0,0.0,0.0,5.21,7.44,12.64,0.03,0.45,0.06
9,1,1975,14,Chhattisgarh,Durg,598.3,454.0,758.82,49.2,27.8,...,1750.00,0.0,0.0,0.0,5.11,7.86,12.97,0.05,0.52,0.08


In [ ]:
import pandas as pd
import json
import requests
import types
from botocore.client import Config
import ibm_boto3
from geopy.geocoders import Nominatim
import re
from sklearn.linear_model import LinearRegression
import numpy as np

# ==============================================================================
# SECTION 0: LOAD DATA AND TRAIN AI MODELS
# ==============================================================================

def load_data_from_cos():
    """
    Connects to IBM Cloud Object Storage and loads the CSV data into a pandas DataFrame.
    """
    print("--- [Data Loading] Connecting to IBM Cloud Object Storage... ---")
    
    # IMPORTANT: Replace these placeholders with your actual IBM Cloud credentials.
    # It is highly recommended to use environment variables for security in a real application.
    IBM_API_KEY_ID = 'jofgetXqPPBxZ9HDeYpRH87wrmKIPt_CgHc2vd92LVPQ' # <-- Replace with your API Key ID
    ENDPOINT_URL = 'https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud' # <-- Replace with your endpoint URL
    BUCKET_NAME = 'aiagent-donotdelete-pr-hhlreaqgqptmqx' # <-- Replace with your bucket name
    OBJECT_KEY = 'ICRISAT-District Level Data.csv'

    cos_client = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=IBM_API_KEY_ID,
        ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
        config=Config(signature_version='oauth'),
        endpoint_url=ENDPOINT_URL)

    try:
        body = cos_client.get_object(Bucket=BUCKET_NAME, Key=OBJECT_KEY)['Body']
        # A more robust way to read the streaming body
        df_data = pd.read_csv(body)
        print("--- [Data Loading] Successfully loaded data from the cloud. ---")
        return df_data
    except Exception as e:
        print(f"--- [Data Loading] CRITICAL ERROR: Could not load data from IBM Cloud. ---")
        print(f"Error details: {e}")
        return None

def train_price_prediction_model(df):
    """
    **AI MODEL TRAINING**
    Trains a linear regression model for each crop to predict price.
    Since the dataset lacks historical prices, we simulate a realistic price trend
    for training purposes. This is a key step to enable AI-based prediction.
    """
    if df is None:
        return None
    print("--- [AI Training] Training price prediction models... ---")
    models = {}
    crop_yield_columns = {
        'wheat': 'WHEAT YIELD (Kg per ha)',
        'rice': 'RICE YIELD (Kg per ha)',
        'sugarcane': 'SUGARCANE YIELD (Kg per ha)'
    }

    for crop, yield_col in crop_yield_columns.items():
        try:
            # Prepare data for the specific crop
            crop_df = df[['Year', yield_col]].dropna()
            crop_df = crop_df[crop_df[yield_col] > 0]

            if crop_df.empty:
                print(f"--- [AI Training] Warning: No data available for '{crop}', skipping model training.")
                continue

            # Simulate historical price data for training
            base_price = {'wheat': 1500, 'rice': 1600, 'sugarcane': 250}.get(crop, 1000)
            yield_factor = 0.1
            year_factor = 20
            
            crop_df['Simulated_Price'] = base_price + \
                                         (crop_df[yield_col] * yield_factor) + \
                                         ((crop_df['Year'] - crop_df['Year'].min()) * year_factor)

            X = crop_df[['Year', yield_col]]
            y = crop_df['Simulated_Price']

            model = LinearRegression()
            model.fit(X, y)
            models[crop] = model
            print(f"--- [AI Training] Model for '{crop}' trained successfully.")
        except KeyError:
            print(f"--- [AI Training] Warning: Column '{yield_col}' not found, skipping model for '{crop}'.")
        except Exception as e:
            print(f"--- [AI Training] Error training model for '{crop}': {e}")

    return models

# Load data and train models at startup
df_data = load_data_from_cos()
price_prediction_models = train_price_prediction_model(df_data)

# --- API Keys and Configuration ---
IBM_WATSONX_API_KEY = "YOUR_IBM_CLOUD_API_KEY"
IBM_WATSONX_URL = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-29"

# ==============================================================================
# SECTION 1: RETRIEVAL TOOLS (The 'R' in RAG)
# ==============================================================================

def get_crop_recommendation(state, district):
    """Analyzes historical data to recommend top-yielding crops for a location."""
    if df_data is None: return "Error: The crop dataset is not available."
    try:
        location_df = df_data[(df_data['State Name'].str.lower() == state.lower()) & (df_data['Dist Name'].str.lower() == district.lower())]
        if location_df.empty: return f"No historical crop data found for {district}, {state}."
        yield_columns = [col for col in df_data.columns if 'YIELD' in col]
        avg_yields = location_df[yield_columns].mean().dropna()
        avg_yields = avg_yields[avg_yields > 0]
        if avg_yields.empty: return f"Could not determine top crops for {district}, {state}."
        avg_yields.index = avg_yields.index.str.replace(' YIELD (Kg per ha)', '').str.title()
        top_5_crops = avg_yields.sort_values(ascending=False).head(5)
        recommendations = "\n".join([f"- {crop}: {int(yield_val)} kg/ha" for crop, yield_val in top_5_crops.items()])
        return f"Historical data for {district}, {state} suggests these high-yield crops:\n{recommendations}"
    except Exception as e: return f"An error occurred during crop analysis: {e}"

def predict_mandi_price(crop, state, district):
    """
    **AI PREDICTION FUNCTION**
    Uses the trained model to predict the mandi price for the next year.
    """
    model = price_prediction_models.get(crop)
    if not model:
        return f"No price prediction model available for {crop}.", None

    if df_data is None:
        return "Error: Crop dataset not available for price prediction.", None
    
    try:
        location_df = df_data[(df_data['State Name'].str.lower() == state.lower()) & (df_data['Dist Name'].str.lower() == district.lower())]
        if location_df.empty: return f"No historical data for {district} to predict price.", None
        
        yield_col = [col for col in df_data.columns if crop.upper() in col and 'YIELD' in col][0]
        avg_yield = location_df[yield_col].mean()
        if pd.isna(avg_yield): return f"Cannot predict price due to missing yield data for {crop} in {district}.", None

        next_year = df_data['Year'].max() + 1
        prediction_features = np.array([[next_year, avg_yield]])
        predicted_price = model.predict(prediction_features)[0]

        return f"AI-Predicted mandi price for {crop.title()} in {district} for next year is ₹{predicted_price:,.0f}/Quintal.", predicted_price
    except Exception as e:
        return f"An error occurred during price prediction: {e}", None

def get_soil_data(district, state):
    """Fetches soil properties from ISRIC SoilGrids API using geocoding."""
    try:
        # It's good practice to specify a unique user_agent for geopy
        geolocator = Nominatim(user_agent="smart_farming_agent_v5_1")
        location = geolocator.geocode(f"{district}, {state}, India")
        if not location: return "Could not geocode the location to fetch soil data."
        lat, lon = location.latitude, location.longitude
        url = f"https://rest.isric.org/soilgrids/v2.0/properties/query?lon={lon}&lat={lat}&property=phh2o&property=soc&depth=0-5cm&value=mean"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        properties = data['properties']['layers']
        ph_data = properties[0]['depths'][0]['values']['mean'] / 10.0
        soc_data = properties[1]['depths'][0]['values']['mean'] / 10.0
        return f"Soil data for {district}: Average pH is {ph_data:.2f}. Average Soil Organic Carbon is {soc_data:.2f} g/kg."
    except Exception as e: return f"Could not fetch soil data. Error: {e}"

def get_pest_control_advice(crop):
    """Provides pest control advice from an expanded knowledge base."""
    advice = {"rice": "For Stem Borer, apply Fipronil 0.3% GR. For Leaf Folder, spray Chlorantraniliprole.", "wheat": "For Aphids, spray Imidacloprid. For Termites, use seed treatment.", "sugarcane": "For Early Shoot Borer, use Chlorantraniliprole 0.4% GR. For Pyrilla, conserve natural predators."}
    return advice.get(crop.lower(), f"No specific pest control advice found for {crop}.")

def get_cultivation_cost(crop):
    """Provides an estimated breakdown of cultivation costs per acre."""
    costs = {"wheat": {"Total Estimated Cost": 19500}, "rice": {"Total Estimated Cost": 25500}, "sugarcane": {"Total Estimated Cost": 43000}}
    cost_data = costs.get(crop.lower())
    if not cost_data: return f"No cost estimation data for {crop}.", None
    return f"Estimated cost of cultivation for {crop.title()} is ₹{cost_data['Total Estimated Cost']:,} per acre.", cost_data['Total Estimated Cost']

def get_profit_prediction(crop, state, district):
    """Calculates estimated revenue and profit based on yield, AI-predicted price, and cost."""
    if df_data is None: return "Error: Crop dataset not available for profit prediction."
    try:
        location_df = df_data[(df_data['State Name'].str.lower() == state.lower()) & (df_data['Dist Name'].str.lower() == district.lower())]
        if location_df.empty: return f"No historical data for {district} to predict profit."
        yield_col = [col for col in df_data.columns if crop.upper() in col and 'YIELD' in col][0]
        avg_yield_kg_per_ha = location_df[yield_col].mean()
        if pd.isna(avg_yield_kg_per_ha) or avg_yield_kg_per_ha == 0: return f"Not enough yield data for {crop} to predict profit."
    except Exception as e: return f"Error getting yield data: {e}"

    price_text, predicted_price = predict_mandi_price(crop, state, district)
    if predicted_price is None: return price_text

    cost_text, cost_per_acre = get_cultivation_cost(crop)
    if cost_per_acre is None: return cost_text

    avg_yield_quintal_per_ha = avg_yield_kg_per_ha / 100
    cost_per_ha = cost_per_acre / 0.404686
    gross_revenue_per_ha = avg_yield_quintal_per_ha * predicted_price
    net_profit_per_ha = gross_revenue_per_ha - cost_per_ha

    return f"""
Profit Prediction for {crop.title()} in {district}, {state}:
- Average Yield: {int(avg_yield_kg_per_ha)} kg/ha
- AI-Predicted Mandi Price: ₹{predicted_price:,.0f}/Quintal
- Gross Revenue (Est.): ₹{gross_revenue_per_ha:,.0f}/ha
- Cultivation Cost (Est.): ₹{cost_per_ha:,.0f}/ha
--------------------------------------------------
- **Net Profit (Est.): ₹{net_profit_per_ha:,.0f}/ha**
"""

# ==============================================================================
# SECTION 2: LANGUAGE AND ROUTING
# ==============================================================================

def route_query(query, state, district):
    """Analyzes the query to determine which retrieval tool to use."""
    query_lower = query.lower()
    crop_match = re.search(r'\b(wheat|rice|sugarcane|tomato|potato)\b', query_lower)
    crop = crop_match.group(1) if crop_match else "wheat"

    if "price" in query_lower or "rate" in query_lower:
        price_text, _ = predict_mandi_price(crop, state, district)
        return price_text
    elif "profit" in query_lower or "income" in query_lower:
        return get_profit_prediction(crop, state, district)
    elif "cost" in query_lower or "investment" in query_lower:
        cost_breakdown, _ = get_cultivation_cost(crop)
        return cost_breakdown
    elif "crop" in query_lower or "best" in query_lower:
        return get_crop_recommendation(state, district)
    elif "soil" in query_lower:
        return get_soil_data(district, state)
    elif "pest" in query_lower:
        return get_pest_control_advice(crop)
    else:
        return "I can provide information on crop recommendations, AI-based price/profit predictions, soil, and pest control. Please ask a specific question."

# ==============================================================================
# SECTION 3: GENERATION (The 'G' in RAG)
# ==============================================================================

def query_ibm_granite(user_question, context, state, district):
    """**MOCK FUNCTION** Constructs the prompt for the IBM Granite LLM."""
    return f"Namaste! Here is the information you requested for {district}. {context}"

# ==============================================================================
# SECTION 4: MAIN APPLICATION LOGIC
# ==============================================================================

def answer_farmer_question(query_text, state, district):
    """Main function to process a farmer's query."""
    print(f"\n--- [Processing Query: '{query_text}'] ---")
    try:
        retrieved_context = route_query(query_text, state, district)
        final_answer = query_ibm_granite(query_text, retrieved_context, state, district)
        print(f"\n>>> FINAL ANSWER:\n{final_answer}\n")
        return final_answer
    except Exception as e:
        print(f"\n>>> An unexpected error occurred: {e}\nPlease try again.")
        return None

def main_loop():
    """Main interactive loop for the agent."""
    if df_data is None or not price_prediction_models:
        print("\nCould not start agent: data loading or AI model training failed.")
        return

    print("="*60)
    print("     Welcome to the AI Smart Farming Agent (V5.1 - Stable)")
    print("="*60)
    user_state = input("Please enter your state: ").strip()
    user_district = input("Please enter your district: ").strip()
    print(f"\nLocation set to: District - {user_district}, State - {user_state}")
    print("Example: 'What is the predicted profit for wheat?' or 'exit' to quit.")
    print("-" * 60)

    while True:
        query = input("Your question: ").strip()
        if query.lower() == 'exit':
            print("Thank you for using the Smart Farming Agent. Goodbye!")
            break
        if not query:
            continue
        
        answer_farmer_question(query, user_state, user_district)

if __name__ == "__main__":
    main_loop()


--- [Data Loading] Connecting to IBM Cloud Object Storage... ---
--- [Data Loading] Successfully loaded data from the cloud. ---
--- [AI Training] Training price prediction models... ---
--- [AI Training] Model for 'wheat' trained successfully.
--- [AI Training] Model for 'rice' trained successfully.
--- [AI Training] Model for 'sugarcane' trained successfully.
     Welcome to the AI Smart Farming Agent (V5.1 - Stable)


Please enter your state:  Uttar Pradesh
Please enter your district:  Jaunpur



Location set to: District - Jaunpur, State - Uttar Pradesh
Example: 'What is the predicted profit for wheat?' or 'exit' to quit.
------------------------------------------------------------


Your question:  What is the predicted profit for wheat?


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


...

Your question:  What is the predicted profit for tomato?



--- [Processing Query: 'What is the predicted profit for tomato?'] ---

>>> FINAL ANSWER:
Namaste! Here is the information you requested for Jaunpur. Error getting yield data: list index out of range



Your question:  What is the predicted profit for rice?


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(



--- [Processing Query: 'What is the predicted profit for rice?'] ---

>>> FINAL ANSWER:
Namaste! Here is the information you requested for Jaunpur. 
Profit Prediction for Rice in Jaunpur, Uttar Pradesh:
- Average Yield: 1525 kg/ha
- AI-Predicted Mandi Price: ₹2,793/Quintal
- Gross Revenue (Est.): ₹42,598/ha
- Cultivation Cost (Est.): ₹63,012/ha
--------------------------------------------------
- **Net Profit (Est.): ₹-20,414/ha**


